## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config_Copy1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kununurra,AU,-15.7667,128.7333,85.98,21,34,17.27,scattered clouds
1,1,Mareeba,AU,-17.0000,145.4333,75.43,71,12,6.53,few clouds
2,2,Prince Rupert,CA,54.3161,-130.3201,59.09,74,40,8.05,light rain
3,3,Oriximina,BR,-1.7656,-55.8661,74.82,90,30,2.26,scattered clouds
4,4,Ravenna,IT,44.4167,11.9833,83.08,67,0,7.25,clear sky
5,5,Geraldton,AU,-28.7667,114.6000,59.41,44,0,19.57,clear sky
6,6,Orje,NO,59.4802,11.6602,54.05,95,52,4.07,broken clouds
7,7,Palmar,CR,9.8728,-84.4553,77.22,97,100,1.92,overcast clouds
8,8,Kaeo,NZ,-35.1000,173.7833,61.32,70,60,7.70,broken clouds
9,9,Novoagansk,RU,61.9449,76.6625,56.84,78,14,7.83,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kununurra,AU,-15.7667,128.7333,85.98,21,34,17.27,scattered clouds
1,1,Mareeba,AU,-17.0000,145.4333,75.43,71,12,6.53,few clouds
4,4,Ravenna,IT,44.4167,11.9833,83.08,67,0,7.25,clear sky
7,7,Palmar,CR,9.8728,-84.4553,77.22,97,100,1.92,overcast clouds
14,14,Tukrah,LY,32.5341,20.5791,78.28,61,0,8.03,clear sky
21,21,Marsaxlokk,MT,35.8419,14.5431,81.09,83,0,2.30,clear sky
23,23,Kapaa,US,22.0752,-159.3190,86.32,67,1,1.99,clear sky
24,24,Hithadhoo,MV,-0.6000,73.0833,82.67,69,100,13.76,overcast clouds
29,29,Atuona,PF,-9.8000,-139.0333,77.99,66,16,13.15,few clouds
33,33,Alofi,NU,-19.0595,-169.9187,75.09,83,92,16.11,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                230
City                   230
Country                230
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Current Description    230
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kununurra,AU,85.98,scattered clouds,-15.7667,128.7333,
1,Mareeba,AU,75.43,few clouds,-17.0000,145.4333,
4,Ravenna,IT,83.08,clear sky,44.4167,11.9833,
7,Palmar,CR,77.22,overcast clouds,9.8728,-84.4553,
14,Tukrah,LY,78.28,clear sky,32.5341,20.5791,
21,Marsaxlokk,MT,81.09,clear sky,35.8419,14.5431,
23,Kapaa,US,86.32,clear sky,22.0752,-159.3190,
24,Hithadhoo,MV,82.67,overcast clouds,-0.6000,73.0833,
29,Atuona,PF,77.99,few clouds,-9.8000,-139.0333,
33,Alofi,NU,75.09,overcast clouds,-19.0595,-169.9187,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json() 
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")    


Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kununurra,AU,85.98,scattered clouds,-15.7667,128.7333,Hotel Kununurra
1,Mareeba,AU,75.43,few clouds,-17.0000,145.4333,Trinity Plains Tourist Park
4,Ravenna,IT,83.08,clear sky,44.4167,11.9833,Hotel Tatì
7,Palmar,CR,77.22,overcast clouds,9.8728,-84.4553,Casa Victoria y Anabella
14,Tukrah,LY,78.28,clear sky,32.5341,20.5791,Lagrota restaurant


In [10]:
# Drop the rows where there is no Hotel Name.
hotel_df ['Hotel Name'].replace(' ', np.nan, inplace=True)
clean_hotel_df= hotel_df.dropna()

##### 7. Drop the rows where there is no Hotel Name.
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))